In [19]:
from langgraph.graph import StateGraph ,START ,END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI as CGA
import re

In [20]:
load_dotenv()

True

In [21]:
model = CGA(model = 'gemini-2.0-flash')

In [22]:
class llmstate(TypedDict):
    topic : str
    outline : str
    blog :str
    score : str

In [23]:
def createoutline(state : llmstate) -> llmstate:
    topic = state['topic']
    prompt = f"Generate a detailed outline for a blog for this topic -> {topic}"
    outline = model.invoke(prompt)
    state['outline'] = outline.content
    return state

In [24]:
def createblog(state : llmstate) -> llmstate:
    topic = state['topic']
    outline = state['outline']
    prompt = f'Write a detailed informative blog , you are given topic and outline\n {topic} {outline}'
    blog = model.invoke(prompt)
    state['blog'] = blog.content
    return state

In [25]:
def evaluator(state : llmstate) -> llmstate:
    topic = state['topic']
    outline = state['outline']
    blog = state['blog']
    prompt = f"""Give only a number between 0 and 100 as score.
Do not provide explanation or justification.
Score the blog based on the topic and outline.

TOPIC: {topic}
OUTLINE: {outline}
BLOG: {blog}
"""

    score_response = model.invoke(prompt)
    raw_output = score_response.content

    # Extract the first number between 0 and 100
    match = re.search(r'\b(100|[1-9]?[0-9])\b', raw_output)
    score = int(match.group(0)) if match else None

    state['score'] = score

    return state


In [26]:
#defining the graph
graph = StateGraph(llmstate)

#defining the node
graph.add_node('createoutline' , createoutline)
graph.add_node('blogcreated' , createblog)
graph.add_node('Evaluator' , evaluator)

#joining the edges
graph.add_edge(START , 'createoutline')
graph.add_edge('createoutline' , 'blogcreated')
graph.add_edge('blogcreated' ,'Evaluator')
graph.add_edge('Evaluator' , END)

#compiling
workflow = graph.compile()

In [27]:
inital_state = {'topic' : 'Rise of trading in Silk route'}
final_state = workflow.invoke(inital_state)


In [31]:
print(final_state['blog'])

## The Rise of Trading in the Silk Road: A Journey Through History and Commerce

**(I. Introduction: The Silk Road - More Than Just Silk)**

**(A. Hook)** Imagine holding a piece of shimmering silk, woven with intricate designs and dyed with vibrant colors. This fabric, once worth more than its weight in gold, traveled thousands of miles across treacherous landscapes, carrying with it not only its beauty but also the seeds of cultural exchange and economic prosperity. This is the legacy of the Silk Road.

**(B. Defining the Silk Road)** The Silk Road wasn't a single road, but rather a sprawling network of trade routes that connected the East and West for centuries. Spanning from East Asia, primarily China, across Central Asia, the Middle East, and ultimately reaching the Mediterranean and even Europe, this intricate web facilitated the flow of goods, ideas, and cultures.

**(C. Beyond Silk)** While silk was undoubtedly a prized commodity, the Silk Road was far more than just a conduit 

In [30]:
print(final_state['score'])

92
